In [1]:
# Referenced: https://github.com/vishnubob/python-midi

In [4]:
# # Update midi package
# !pip install git+https://github.com/vishnubob/python-midi@feature/python3
# !git clone -b feature/python3 https://github.com/vishnubob/python-midi.git
# !pip install --upgrade git+https://github.com/louisabraham/python3-midi

There are 3 parts to this notebook.

1) Functions

2) Extracting the melody from the midi file. (`solo_mid` function)

3) Chopping up the solo midi to chunks with the same number of states. (`split_midi` function)

# Functions

In [31]:
import mido # https://github.com/mido/mido
import midi # https://github.com/vishnubob/python-midi
import random
import numpy as np
import scipy as sp
from collections import defaultdict, Counter
import itertools
import statistics as stats

In [32]:
def solo_midi(og_filename, new_filename, c_num):
    """
    Input: `og_filename` = the name of the file for the original midi file,
           `og_filename` = the name of the new midi file with the isolated channel,
           `c_num` = the number of the channel that we want to isolate
    """
    # Read the file
    music = midi.read_midifile(og_filename)

    # Extract the violin solo in channel c_num and save it
    # Instantiate a MIDI Pattern (contains a list of tracks)
    pattern = midi.Pattern()
    # Append the track to the pattern
    pattern.append(music[0]) # info on tempo
    if c_num > 0:
        pattern.append(music[c_num])
        
    # convert to violin and move everything to channel 0
    for line in pattern:
        for event in line:
            if type(event) == midi.events.ProgramChangeEvent:
                event.data = [40]
            if type(event) == midi.events.NoteOffEvent or type(event) == midi.events.NoteOnEvent:
                event.channel = 0
            if type(event) == midi.events.ControlChangeEvent:
                event.channel = 0
    
    # Save the pattern to disk
    midi.write_midifile(new_filename, pattern)

    # Keep same ticks per beat
    mid_new = mido.MidiFile(new_filename)
    mid_og =  mido.MidiFile(og_filename)

    mid_new.ticks_per_beat = mid_og.ticks_per_beat

    mid_new.save(new_filename)

In [33]:
def remove_zeros(states, k):
    """Removes long pauses in the piece (only allows for 0 to appear k times consecutively)
    """
    new_states = []
    count = 0 # counts the number of consecutive 0's
    
    for s in states:
        if s == 0: # if current state is a 0
            count += 1
            if count <= k:
                new_states.append(s)
        else:
            count = 0 # reset counter
            new_states.append(s)
                
    return new_states

In [34]:
def midi_to_unigram(FILE_NAME, BASE_TICK, LOWEST_NOTE):
    # Initializations
    current_state = 0;

    # Open the MIDI file
    pattern = midi.read_midifile( FILE_NAME );
    
    # Initialize list to store unigrams
    current_states = []

    # Read the file for note on and off events
    for line in pattern:
        for event in line:
            if type(event) == midi.events.NoteOffEvent or type(event) == midi.events.NoteOnEvent:

                # Write the state enough number of times
                for x in range(0, int( round( float(event.tick) / float(BASE_TICK) ) ) ):
                    current_states.append(current_state)
                
                if type(event) == midi.events.NoteOffEvent or event.get_velocity() <= 0:
                    if event.get_pitch() == current_state + LOWEST_NOTE - 1:
                        current_state = 0;
                else:
                    current_state = event.get_pitch() - LOWEST_NOTE + 1;
    
    return current_states

In [35]:
def unigram_to_midi(unigrams, bpm, lowest_note, instrument, base_tick, file_name):
    BPM = bpm;
    LOWEST_NOTE = lowest_note; # Decide how much transpose
    INSTRUMENT = instrument; # Choose the instrument rendering the music
    BASE_TICK = base_tick;
    FILE_NAME = file_name;

    prev_unigram = 0; # No sound to start with
    tick_count = 0;

    pattern = midi.Pattern();

    track = midi.Track();
    pattern.append( track );

    set_instrument_event = midi.ProgramChangeEvent();
    set_instrument_event.set_value( INSTRUMENT );
    track.append( set_instrument_event );

    set_tempo_event = midi.SetTempoEvent();
    set_tempo_event.set_bpm( BPM );
    track.append( set_tempo_event );

    for unigram_string in unigrams:
        tick_count = tick_count + 1;
        unigram = int( unigram_string );
        if unigram != prev_unigram:
            note = unigram + ( LOWEST_NOTE - 1 );
            previous_note = prev_unigram + ( LOWEST_NOTE - 1 );

            if prev_unigram != 0: # First turn off any notes that are playing already
                event = midi.NoteOffEvent( tick = tick_count * BASE_TICK, velocity = 0, pitch = previous_note );
                track.append( event );
                tick_count = 0;

            if unigram != 0: # We have a new note, so play it
                event = midi.NoteOnEvent( tick = tick_count * BASE_TICK, velocity = 120, pitch = note );
                track.append( event );
                tick_count = 0;

        prev_unigram = unigram


    eot = midi.EndOfTrackEvent( tick = 1 );
    track.append( eot );
    

    midi.write_midifile(FILE_NAME, pattern );

In [51]:
def split_midi(MIDI_FILE, num_notes):
    """
    Input: name of midi file to split up; number of notes in each split
    WARNING: do not play the outputted file. This is only for training the model
    """
    
    # get necessary information, like lowest note, base tick, bpm
    pattern = midi.read_midifile( MIDI_FILE );
    notes_list = []
    ticks_list = [] 
    bpm_list = []
    INSTRUMENT = 40
    for line in pattern:
        for event in line:
            # get instrument
            if type(event) == midi.events.ProgramChangeEvent:
                INSTRUMENT = event.get_value()
            
            # get BPM
            if type(event) == midi.events.SetTempoEvent:
                bpm_list.append(int(event.get_bpm()))

            if type(event) == midi.events.NoteOffEvent or type(event) == midi.events.NoteOnEvent:
                notes_list.append(event.get_pitch())
                ticks_list.append(event.tick)

    if not bpm_list:
        BPM = 100
    else:
        BPM = stats.mode(bpm_list)
    
    notes = list(set(notes_list))
    notes.sort()
    ticks = list(set(ticks_list))
    ticks.sort()
    
    # get number corresponding to lowest note in the piece
    LOWEST_NOTE = notes[0]
    
    # get lowest tick interval (interval all ticks can be evenly divided by)
    BASE_TICK = ticks[1]
    
    # midi to unigram
    current_states = remove_zeros(midi_to_unigram(MIDI_FILE, BASE_TICK, LOWEST_NOTE), 2)
    PIECE_LENGTH = len(current_states)
    print(PIECE_LENGTH)
    
    if PIECE_LENGTH > num_notes:
        for i in range(PIECE_LENGTH//num_notes):
            unigram_to_midi(current_states[i*num_notes:(i+1)*num_notes], BPM, LOWEST_NOTE, INSTRUMENT, BASE_TICK,
                           MIDI_FILE[:-4] + '_split' +str(i)+'.mid')
    else:
        midi.write_midifile(MIDI_FILE[:-4] + '_split0.mid', pattern );

## Bach

In [227]:
og_filename = 'midi/bwv1003/vs2-1gra.mid'
new_filename = 'midi/bach-solo/bwv1003_1_solo.mid'
c_num = 1

music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

###
og_filename = 'midi/bwv1003/vs2-2fug.mid'
new_filename = 'midi/bach-solo/bwv1003_2_solo.mid'
c_num = 1

music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

###
og_filename = 'midi/bwv1003/vs2-3and.mid'
new_filename = 'midi/bach-solo/bwv1003_3_solo.mid'
c_num = 1

music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

###
og_filename = 'midi/bwv1003/vs2-4alg.mid'
new_filename = 'midi/bach-solo/bwv1003_4_solo.mid'
c_num = 1

music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

In [228]:
og_filename = 'midi/bwv1041/bvc1_1.mid'
new_filename = 'midi/bach-solo/bwv1041_1_solo.mid'
c_num = 1

music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

###
og_filename = 'midi/bwv1041/bvc1_2.mid'
new_filename = 'midi/bach-solo/bwv1041_2_solo.mid'
c_num = 1

music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

###
og_filename = 'midi/bwv1041/bvc1_3.mid'
new_filename = 'midi/bach-solo/bwv1041_3_solo.mid'
c_num = 1

music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

In [229]:
og_filename = 'midi/bwv1044/btc_1.mid'
new_filename = 'midi/bach-solo/bwv1044_1_solo.mid'
c_num = 1

music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

###
og_filename = 'midi/bwv1044/btc_2.mid'
new_filename = 'midi/bach-solo/bwv1044_2_solo.mid'
c_num = 1

music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

###
og_filename = 'midi/bwv1044/btc_3.mid'
new_filename = 'midi/bach-solo/bwv1044_3_solo.mid'
c_num = 1

music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

# Mozart

In [230]:
og_filename = 'midi/mozart/kv310_1.mid'
new_filename = 'midi/mozart-solo/kv310_1_solo.mid'
c_num = 1

music = midi.read_midifile(og_filename)

solo_midi(og_filename, new_filename, c_num)

In [231]:
og_filename = 'midi/mozart/kv310_2.mid'
new_filename = 'midi/mozart-solo/kv310_2_solo.mid'
c_num = 1

music = midi.read_midifile(og_filename)

solo_midi(og_filename, new_filename, c_num)

In [232]:
og_filename = 'midi/mozart/kv310_3.mid'
new_filename = 'midi/mozart-solo/kv310_3_solo.mid'
c_num = 1

music = midi.read_midifile(og_filename)

solo_midi(og_filename, new_filename, c_num)

In [233]:
for line in music:
    for event in line:
        print(event)

midi.TimeSignatureEvent(tick=0, data=[2, 2, 96, 8])
midi.SequencerSpecificEvent(tick=0, data=[0, 0, 65])
midi.SetTempoEvent(tick=0, data=[4, 249, 198])
midi.EndOfTrackEvent(tick=0, data=[])
midi.ControlChangeEvent(tick=0, channel=0, data=[0, 0])
midi.ProgramChangeEvent(tick=0, channel=0, data=[0])
midi.ControlChangeEvent(tick=0, channel=0, data=[10, 84])
midi.ControlChangeEvent(tick=0, channel=0, data=[91, 54])
midi.ControlChangeEvent(tick=0, channel=0, data=[93, 56])
midi.KeySignatureEvent(tick=0, data=[0, 0])
midi.ControlChangeEvent(tick=0, channel=0, data=[121, 0])
midi.ControlChangeEvent(tick=96, channel=0, data=[10, 73])
midi.ControlChangeEvent(tick=48, channel=0, data=[91, 63])
midi.ControlChangeEvent(tick=48, channel=0, data=[93, 40])
midi.ProgramChangeEvent(tick=96, channel=0, data=[0])
midi.NoteOnEvent(tick=480, channel=0, data=[72, 60])
midi.NoteOnEvent(tick=0, channel=0, data=[69, 60])
midi.NoteOffEvent(tick=572, channel=0, data=[72, 0])
midi.NoteOffEvent(tick=0, channel=0, 

midi.NoteOnEvent(tick=4, channel=1, data=[62, 82])
midi.NoteOffEvent(tick=188, channel=1, data=[62, 0])
midi.NoteOnEvent(tick=4, channel=1, data=[60, 82])
midi.NoteOffEvent(tick=188, channel=1, data=[60, 0])
midi.NoteOnEvent(tick=196, channel=1, data=[53, 75])
midi.NoteOffEvent(tick=188, channel=1, data=[53, 0])
midi.NoteOnEvent(tick=4, channel=1, data=[65, 75])
midi.NoteOffEvent(tick=188, channel=1, data=[65, 0])
midi.NoteOnEvent(tick=4, channel=1, data=[64, 75])
midi.NoteOffEvent(tick=188, channel=1, data=[64, 0])
midi.NoteOnEvent(tick=4, channel=1, data=[62, 75])
midi.NoteOffEvent(tick=380, channel=1, data=[62, 0])
midi.NoteOnEvent(tick=4, channel=1, data=[64, 75])
midi.NoteOffEvent(tick=380, channel=1, data=[64, 0])
midi.NoteOnEvent(tick=4, channel=1, data=[65, 75])
midi.NoteOffEvent(tick=380, channel=1, data=[65, 0])
midi.NoteOnEvent(tick=580, channel=1, data=[50, 75])
midi.NoteOffEvent(tick=188, channel=1, data=[50, 0])
midi.NoteOnEvent(tick=4, channel=1, data=[62, 75])
midi.Note

midi.NoteOffEvent(tick=0, channel=1, data=[38, 0])
midi.NoteOnEvent(tick=4, channel=1, data=[40, 75])
midi.NoteOnEvent(tick=0, channel=1, data=[36, 75])
midi.NoteOffEvent(tick=188, channel=1, data=[40, 0])
midi.NoteOffEvent(tick=0, channel=1, data=[36, 0])
midi.NoteOnEvent(tick=4, channel=1, data=[38, 75])
midi.NoteOnEvent(tick=0, channel=1, data=[35, 75])
midi.NoteOffEvent(tick=380, channel=1, data=[38, 0])
midi.NoteOffEvent(tick=0, channel=1, data=[35, 0])
midi.NoteOnEvent(tick=4, channel=1, data=[52, 75])
midi.NoteOnEvent(tick=0, channel=1, data=[44, 75])
midi.NoteOffEvent(tick=380, channel=1, data=[52, 0])
midi.NoteOffEvent(tick=0, channel=1, data=[44, 0])
midi.NoteOnEvent(tick=4, channel=1, data=[53, 75])
midi.NoteOnEvent(tick=0, channel=1, data=[45, 75])
midi.NoteOffEvent(tick=380, channel=1, data=[53, 0])
midi.NoteOffEvent(tick=0, channel=1, data=[45, 0])
midi.NoteOnEvent(tick=4, channel=1, data=[50, 75])
midi.NoteOffEvent(tick=380, channel=1, data=[50, 0])
midi.NoteOnEvent(tick

## Vivaldi

In [234]:
og_filename = 'midi/vivaldi/v_concerto.mid'
new_filename = 'midi/vivaldi-solo/v_concerto_solo.mid'
c_num = 0

music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

## Tchaikovsky

In [235]:
og_filename = 'midi/tchaikovsky/old_french_song.mid'
new_filename = 'midi/tchaikovsky-solo/old_french_song_solo.mid'
c_num = 1

music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

In [236]:
og_filename = 'midi/tchaikovsky/trio1.mid'
new_filename = 'midi/tchaikovsky-solo/trio1_solo.mid'
c_num = 1

music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

In [237]:
og_filename = 'midi/tchaikovsky/trio2.mid'
new_filename = 'midi/tchaikovsky-solo/trio2_solo.mid'
c_num = 1

music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

## Paganini

In [238]:
og_filename = 'midi/paganini/Capriccio5.mid'
new_filename = 'midi/paganini-solo/Capriccio5_solo.mid'
c_num = 1

music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

In [239]:
og_filename = 'midi/paganini/Capriccio7.mid'
new_filename = 'midi/paganini-solo/Capriccio7_solo.mid'
c_num = 1

music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

In [240]:
og_filename = 'midi/paganini/Capriccio24.mid'
new_filename = 'midi/paganini-solo/Capriccio24_solo.mid'
c_num = 1

music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

## Schumann

In [241]:
og_filename = 'midi/schumann/sonata105.mid'
new_filename = 'midi/schumann-solo/sonata105_solo.mid'
c_num = 1

music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

In [242]:
og_filename = 'midi/schumann/stringquart_op41.mid'
new_filename = 'midi/schumann-solo/stringquart_op41_solo.mid'
c_num = 1

music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

In [247]:
og_filename = 'midi/schumann/piano_54.mid'
new_filename = 'midi/schumann-solo/piano_54_solo.mid'
c_num = 2

music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

## Handel

In [243]:
og_filename = 'midi/handel/HWV322a.mid'
new_filename = 'midi/handel-solo/HWV322a_solo.mid'
c_num = 1

music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

###
og_filename = 'midi/handel/HWV322b.mid'
new_filename = 'midi/handel-solo/HWV322b_solo.mid'
c_num = 1

music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

###
og_filename = 'midi/handel/HWV322c.mid'
new_filename = 'midi/handel-solo/HWV322c_solo.mid'
c_num = 1

music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

###
og_filename = 'midi/handel/HWV322d.mid'
new_filename = 'midi/handel-solo/HWV322d_solo.mid'
c_num = 1

music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

## Pozzoli (Piano and not good because everything is mashed into one channel)

In [245]:
# og_filename = 'midi/pozzoli/studio3.mid'
# new_filename = 'midi/pozzoli-solo/studio3_solo.mid'
# c_num = 1

# music = midi.read_midifile(og_filename)
# solo_midi(og_filename, new_filename, c_num)

# More Pieces

In [248]:
# Corelli
og_filename = 'midi/c major/corelli_sonata.mid'
new_filename = 'midi/c major-solo/corelli_sonata_solo.mid'
c_num = 1

music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)



In [253]:
# Mozart

og_filename = 'midi/c major/mozart_kv6_1.mid'
new_filename = 'midi/c major-solo/mozart_kv6_1_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/c major/mozart_kv6_2.mid'
new_filename = 'midi/c major-solo/mozart_kv6_2_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/c major/mozart_kv6_3.mid'
new_filename = 'midi/c major-solo/mozart_kv6_3_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/c major/mozart_kv6_4.mid'
new_filename = 'midi/c major-solo/mozart_kv6_4_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/c major/mozart_kv296.mid'
new_filename = 'midi/c major-solo/mozart_kv296_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

In [255]:
# Paganini

og_filename = 'midi/c major/paganini_11.mid'
new_filename = 'midi/c major-solo/paganini_11_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/c major/paganini_18.mid'
new_filename = 'midi/c major-solo/paganini_18_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

In [256]:
# Albinoni
og_filename = 'midi/c major/albinoni_75a.mid'
new_filename = 'midi/c major-solo/albinoni_75a_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/c major/albinoni_75b.mid'
new_filename = 'midi/c major-solo/albinoni_75b_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/c major/albinoni_75c.mid'
new_filename = 'midi/c major-solo/albinoni_75c_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

In [257]:
og_filename = 'midi/c major/albinoni_99a.mid'
new_filename = 'midi/c major-solo/albinoni_99a_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/c major/albinoni_99b.mid'
new_filename = 'midi/c major-solo/albinoni_99b_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/c major/albinoni_99c.mid'
new_filename = 'midi/c major-solo/albinoni_99c_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

In [260]:
# # Beethoven

# og_filename = 'midi/c major/beethoven_56a.mid'
# new_filename = 'midi/c major-solo/beethoven_56a_solo.mid'
# music = midi.read_midifile(og_filename)
# solo_midi(og_filename, new_filename, c_num)

In [284]:
c_num=9

og_filename = 'midi/c major/beethoven_s1a.mid'
new_filename = 'midi/c major-solo/beethoven_s1a_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/c major/beethoven_s1b.mid'
new_filename = 'midi/c major-solo/beethoven_s1b_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/c major/beethoven_s1c.mid'
new_filename = 'midi/c major-solo/beethoven_s1c_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/c major/beethoven_s1d.mid'
new_filename = 'midi/c major-solo/beethoven_s1d_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

In [14]:
c_num = 1
og_filename = 'midi/c major/beethoven_quartet09-1.mid'
new_filename = 'midi/c major-solo/beethoven_quartet09-1_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/c major/beethoven_quartet09-2.mid'
new_filename = 'midi/c major-solo/beethoven_quartet09-2_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/c major/beethoven_quartet09-3.mid'
new_filename = 'midi/c major-solo/beethoven_quartet09-3_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/c major/beethoven_quartet09-4.mid'
new_filename = 'midi/c major-solo/beethoven_quartet09-4_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

In [287]:
# TARTINI

c_num = 1
og_filename = 'midi/c major/tartini_sonata.mid'
new_filename = 'midi/c major-solo/tartini_sonata_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

In [18]:
# MORE BACH
og_filename = 'midi/baroque/bwv1001a.mid'
new_filename = 'midi/baroque-solo/bwv1001a_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/baroque/bwv1001b.mid'
new_filename = 'midi/baroque-solo/bwv1001b_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/baroque/bwv1001c.mid'
new_filename = 'midi/baroque-solo/bwv1001c_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/baroque/bwv1001d.mid'
new_filename = 'midi/baroque-solo/bwv1001d_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)
######
og_filename = 'midi/baroque/bwv1002a.mid'
new_filename = 'midi/baroque-solo/bwv1002a_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/baroque/bwv1002b.mid'
new_filename = 'midi/baroque-solo/bwv1002b_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/baroque/bwv1002c.mid'
new_filename = 'midi/baroque-solo/bwv1002c_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/baroque/bwv1002d.mid'
new_filename = 'midi/baroque-solo/bwv1002d_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/baroque/bwv1002e.mid'
new_filename = 'midi/baroque-solo/bwv1002e_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/baroque/bwv1002f.mid'
new_filename = 'midi/baroque-solo/bwv1002f_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/baroque/bwv1002g.mid'
new_filename = 'midi/baroque-solo/bwv1002g_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/baroque/bwv1002h.mid'
new_filename = 'midi/baroque-solo/bwv1002h_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

In [15]:
# CHOPIN
c_num = 1
og_filename = 'midi/chopin/chopin_etude_10_2.mid'
new_filename = 'midi/chopin-solo/chopin_etude_10_2_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

In [20]:
og_filename = 'midi/chopin/mazurka-50.mid'
new_filename = 'midi/chopin-solo/mazurka-50_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/chopin/mazurka-51.mid'
new_filename = 'midi/chopin-solo/mazurka-51_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/chopin/mazurka-52.mid'
new_filename = 'midi/chopin-solo/mazurka-52_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/chopin/mazurka07-2.mid'
new_filename = 'midi/chopin-solo/mazurka07-2_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/chopin/mazurka07-5.mid'
new_filename = 'midi/chopin-solo/mazurka07-5_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/chopin/mazurka17-4.mid'
new_filename = 'midi/chopin-solo/mazurka17-4_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/chopin/mazurka24-2.mid'
new_filename = 'midi/chopin-solo/mazurka24-2_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/chopin/mazurka33-3.mid'
new_filename = 'midi/chopin-solo/mazurka33-3_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/chopin/mazurka56-2.mid'
new_filename = 'midi/chopin-solo/mazurka56-2_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/chopin/mazurka59-1.mid'
new_filename = 'midi/chopin-solo/mazurka59-1_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/chopin/mazurka67-3.mid'
new_filename = 'midi/chopin-solo/mazurka67-3_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/chopin/mazurka67-4.mid'
new_filename = 'midi/chopin-solo/mazurka67-4_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/chopin/mazurka68-1.mid'
new_filename = 'midi/chopin-solo/mazurka68-1_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/chopin/mazurka68-2.mid'
new_filename = 'midi/chopin-solo/mazurka68-2_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/chopin/mazurka33-4.mid'
new_filename = 'midi/chopin-solo/mazurka33-4_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/chopin/waltz69-2.mid'
new_filename = 'midi/chopin-solo/waltz69-2_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

In [19]:
# More Paganini
og_filename = 'midi/romantic/capriccio_2.mid'
new_filename = 'midi/romantic-solo/capriccio_2_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/romantic/capriccio_6.mid'
new_filename = 'midi/romantic-solo/capriccio_6_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/romantic/capriccio_10.mid'
new_filename = 'midi/romantic-solo/capriccio_10_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

og_filename = 'midi/romantic/capriccio_16.mid'
new_filename = 'midi/romantic-solo/capriccio_16_solo.mid'
music = midi.read_midifile(og_filename)
solo_midi(og_filename, new_filename, c_num)

# SPLIT MIDI

In [72]:
from os import listdir
from os.path import isfile, join

mypath_b = 'MIDI PERIOD CLASSIFICATION/Baroque/'
b_files = [mypath_b+f for f in listdir(mypath_b) if isfile(join(mypath_b, f)) and f != '.DS_Store']

mypath_r = 'MIDI PERIOD CLASSIFICATION/Romantic/'
r_files = [mypath_r+f for f in listdir(mypath_r) if isfile(join(mypath_r, f))and f != '.DS_Store']

In [78]:
for file in b_files:
    split_midi(file, 2000)
    
for file in r_files:
    split_midi(file, 2000)

770
961
26123
35397
2170
31938
24898
2780
3288
1600
71346
1656
2156
30138
80029
22778
1594
1936
1922
5224
39135
1518
11616
120720
16662
53109
1632
1044
2920
564
51344
11023
46929
576
960
1594
1665
14977
17532
125890
3991
186169
17284
2188
19727
47310
21287
23244
39376
312716
4152
39492
37684
19357
38816
120699
578
786
23672
1006
69588
4862
19617
26792
24073
45108
3305
